# LegoCons(路径,搭建项目名称)    类

- 类初始化时，会运行一次creatItem()
    
- LegoCons.creatItem()
    - 如果没有同名的，则创建文件夹
    - 文件夹下建立：零件总图、video 两个子文件夹
    
- LegoCons.expHTML_lego()
    - 包括有 **makeList()**
    - 导出相应的微信公众号文章后台可左右滑动图片代码，代码文件名为：002长颈鹿_html.txt
    - 相应的照片应该传到腾讯cos上
    
- LegoCons.putTag()
    - 给零件总图下的图片按“步骤列表.xlsx”中的描述添加文字
    - 生成文件名为xxx_tagged.png
    
- LegoCons.makeList()
    - 根据文件夹中的图片，生成排序列表，用于：
        - 供 expHTML_lego 制作公众号后台图片滑动
        - 供 ConsMovie 类将图片合成视频
    - 排序列表导出“步骤列表.xlsx”保存在文件夹内
    
- 举例：
    ```
    mylego=LegoCons('I:\\乐高\\图纸','002长颈鹿')
    mylego.creatItem()
    mylego.makeList()
    mylego.putTag()
    mylego.expHTML_lego()
    ```
    
    
# ConsMovie(路径,搭建项目名称)   类

- 实例化时即运行 **LegoCons.makeList()**
- ConsMovie.expMovie()
    - 导出视频，命名为“002长颈鹿_搭建视频.mp4”

- 举例：
    ```
    myMovie=ConsMovie('I:\\乐高\\图纸','002长颈鹿')
    myMovie.expMovie()
    ```


In [5]:
import os
from moviepy.editor import *
import moviepy.audio.fx.all as afx
import pandas as pd
import numpy as np
from PIL import Image
from PIL import ImageDraw  
from PIL import ImageFont


class LegoCons:
    def __init__(self,pth,consName):
        self.pth=pth
        self.consName=consName
        self.creatItem()
        
    def creatItem(self):
        New=os.path.join(self.pth,self.consName)
        if not os.path.exists(New):
            print('新项目，创建文件夹...\n')
            os.makedirs(New)
            new_totals=os.path.join(self.pth,self.consName,'零件总图')
            new_video=os.path.join(self.pth,self.consName,'video')
            print('文件夹{}创建完成\n'.format(' '+self.consName+' '))
            
            if not os.path.exists(new_totals):
                os.makedirs(new_totals)
                print('文件夹{}创建完成\n'.format(' 零件总图 '))
            if not os.path.exists(new_video):
                os.makedirs(new_video)
                print('文件夹{}创建完成\n'.format(' video '))
        else:
            print('已有项目\n')
            new_totals=os.path.join(self.pth,self.consName,'零件总图')
            new_video=os.path.join(self.pth,self.consName,'video')
            if not os.path.exists(new_totals):
                os.makedirs(new_totals)
                print('文件夹{}创建完成\n'.format(' 零件总图 '))
            if not os.path.exists(new_video):
                os.makedirs(new_video)
                print('文件夹{}创建完成\n'.format(' video '))
          
    def expHTML_lego(self):
        lsts=self.makeList()     
        
        html_pic_addr=''
        _pre_pic='''
        </section>
                        <section style="color: #bfbfbf;padding-top: 10px;padding-bottom: 10px;display: inline-block;width: 100%;box-sizing: border-box;" data-width="75%">
                            <section class="_135editor" data-tools="135编辑器" data-id="95138">
                                <section class="_135editor">
                                    <section style="margin: 1em auto;text-align: center;padding: 5px;border-width: 1px;border-style: solid;border-color: transparent;overflow: hidden;box-sizing: border-box;">
                                        <section class="135brush" data-style="display: inline-block;width: 100%;margin:0;padding:0;" style="white-space: nowrap;overflow-x: scroll;">
        '''


        pre_pic_addr='''
         <img class="" data-ratio="0.75" data-type="jpeg" data-w="1200" data-width="100%" src="https://chuntianhuahua-1257410889.cos.ap-guangzhou.myqcloud.com/legoCons/
         '''
        after_pic_addr='"/>'
        
        for lst in lsts:
            html_pic_addr=html_pic_addr+pre_pic_addr.strip()+lst+after_pic_addr+'\n'
            
        out=_pre_pic+html_pic_addr
        print('生成html...完成\n')
        
        with open(os.path.join(self.pth,self.consName,self.consName+'_html.txt'),'w') as f:
            f.write(out)
        
        print('写入txt...完成\n')
        
    def putTag(self):        
        dirs=os.path.join(self.pth,self.consName,'零件总图')
        df=pd.read_excel(os.path.join(self.pth,self.consName,'步骤列表.xlsx'))
        tag_list=np.array(df.loc[~pd.isna(df['描述'])]).tolist()
        
        font = ImageFont.truetype('C:\Windows\Fonts\msyh.ttc',80)
        for lst in tag_list:
            pth=os.path.join(self.pth,lst[0])
            txt='结构分解图 - '+lst[1]
            print(pth,txt)
            img=Image.open(pth)
            
            draw = ImageDraw.Draw(img)            
            draw.text((200,int(img.size[1]*0.87)), txt, fill = '#95ff67',font=font)
            NewName=pth.replace('.png','_tagged.png')
            img.save(NewName)
#             img.show()       
#             break
      
    def makeList(self):
        n=0
        pngList=[]
        for file in os.listdir(os.path.join(self.pth,self.consName)):
            if file[-3:].lower()=='png' and len(file)<10:
                os.rename(os.path.join(self.pth,self.consName,file),os.path.join(self.pth,self.consName,file.zfill(10)))
                pngList.append(file.zfill(10))
                n+=1
            elif file[-3:].lower()=='png':
                pngList.append(self.consName+"/"+file)
        pngList.sort()
        if n>0:
            print('改名{}个文件'.format(n))
        
        if os.path.exists(os.path.join(self.pth,self.consName,'零件总图')):
#             print('存在零件总图\n')
            totalList=[]
            for file in os.listdir(os.path.join(self.pth,self.consName,'零件总图')):
                if file[-10:].lower()=='tagged.png': #把加标签的放入列表
                    totalList.append(self.consName+'/零件总图/'+file)
        else:
            pass
#             print('无零件总图')
        totalList.sort()
        
        outList=[self.consName+'/'+self.consName[3:]+'.jpg']
        outList.extend(totalList)
        outList.extend(pngList)
        
        df=pd.DataFrame(outList)
        df.columns=['步骤']
        df['描述']=''
        df.to_excel(os.path.join(self.pth,self.consName,'步骤列表.xlsx'),index=False)
        
        print('生成 文件列表...完成\n')
        return outList    
    
class ConsMovie:
    def __init__(self,pth,consName):
        self.bgm='I:\大智小超\公共素材\声音类\legoBGM.mp3'
        self.endV='I:\大智小超\公共素材\视频类\片尾01.mp4'
        self.pth=pth
        self.consName=consName

        a=LegoCons(self.pth,self.consName)
        self.lst=a.makeList()
        
    
    def expMovie(self):
        print('正在剪辑...\n')
        w,h=850,480
        clips=[]
        n=0
        drtn=3
        crstime=1
        for p in self.lst:
            fn=os.path.join(self.pth,p)
            if n==0:
                _img=ImageClip(fn).set_fps(25).set_duration(drtn).resize((w,h))
                clips.append(_img)
                n+=1
            else:
                _img=ImageClip(fn).set_fps(25).set_duration(drtn).resize((w,h)).crossfadein(crstime).set_start((drtn-crstime)*n)
                clips.append(_img)
                n+=1    
                
        #计算正片时间，供logo字幕用
        drt=0
        for c in clips:
            drt=drt+(c.duration-crstime)
        subtitle_drt=drt-clips[0].duration                            
                
        #加上片尾
        print('正在处理片尾...\n')
        endvideo=VideoFileClip(self.endV,target_resolution=(h,w)).set_start((drtn-crstime)*n) # 片尾   分辨率是先写h,再写w  大坑
        clips.append(endvideo)        
        
        print('正在处理字幕...\n')
        text = TextClip(txt="大颗粒积木搭建", fontsize=30, font='Microsoft-YaHei-&-Microsoft-YaHei-UI',color='#95ff67').set_pos((15,15)).set_duration(3)
        clips.append(text)
        text=self.subtitle_right_btm(self.consName[3:],60,w,h)
        clips.append(text)
        text = TextClip(txt='大智小超科学实验室', fontsize=15, \
                 font='Microsoft-YaHei-&-Microsoft-YaHei-UI',color='#9EACC1') \
                .set_pos((685,455)).set_start(clips[0].duration).set_duration(subtitle_drt)        
        clips.append(text)
                
        # finalclip = concatenate_videoclips([_img1,_img2])
        print('正在拼接视频...\n')
        finalclip = CompositeVideoClip(clips)
        
        #加上背景音乐
        print('正在添加背景音乐...\n')
        BGM=AudioFileClip(self.bgm).set_duration(finalclip.duration-endvideo.duration).fx(afx.audio_fadeout,0.8)
        final_audio = CompositeAudioClip([BGM,finalclip.audio])
        mix=finalclip.set_audio(final_audio)
        
        
        out=os.path.join(self.pth,self.consName,self.consName+'_搭建视频.mp4')
        print('正在导出视频：{}...\n'.format(out))
        mix.write_videofile(out)
        print('Done')
        
    def subtitle_right_btm(self,txt,fsz,w,h):
        x=w-len(txt)*fsz-30
        y=h-fsz-30
        text = TextClip(txt=txt, fontsize=fsz, font='Microsoft-YaHei-&-Microsoft-YaHei-UI',color='#95ff67').set_pos((x,y)).set_duration(3)
        return text
    
class help():
    def __init__(self):
        txt='''
        doLego包括两个类：LegoCons和ConsMovie， 
        LegoCons用于生成html格式的步骤图，并保存为txt格式，
        ConsMovie用于将已有的图片剪辑生成视频。
        
        输入：
        doLego.help.helpConsMove()
        doLego.help.helpLegoCons()
        
        '''
        print(txt)
    
    def helpConsMovie(self):
        txt='''
        doLego.ConsMovie(path,name)
        doLego.ConsMovie.expMovie()
        
        example:
        myMovie=ConsMovie('I:\\乐高\\图纸','002长颈鹿')
        myMovie.expMovie()
        '''
        print(txt)
        
    def helpLegoCons(self):
        txt='''
        doLego.LegoCons(path,name)
        doLego.LegoCons.creatItem()
        doLego.LegoCons.expHTML_lego()
        
        example_1 从已有的图片生成公众号文章后台html格式步骤，并保存为txt文件:
        mylego=LegoCons('I:\\乐高\\图纸','003鳄鱼')
        mylego.expHTML_lego()
        
        example_2 新建一个项目文件夹，该文件夹下将包含“零件总图”、“video”两个子文件夹:
        mylego=LegoCons('I:\\乐高\\图纸','004公鸡')
        mylego.expHTML_lego()
        '''
        print(txt)
    
    
if __name__=='__main__':
#     mylego=LegoCons('I:\\乐高\\图纸','002长颈鹿')
#     mylego.creatItem()
#     mylego.makeList()
#     mylego.putTag()
#     mylego.expHTML_lego()

    
    
    myMovie=ConsMovie('I:\\乐高\\图纸','002长颈鹿')
    myMovie.expMovie()

已有项目

生成 文件列表...完成

正在剪辑...

正在处理片尾...

正在处理字幕...

正在拼接视频...

正在添加背景音乐...

正在导出视频：I:\乐高\图纸\002长颈鹿\002长颈鹿_搭建视频.mp4...

Moviepy - Building video I:\乐高\图纸\002长颈鹿\002长颈鹿_搭建视频.mp4.
MoviePy - Writing audio in %s


MoviePy - Done.
Moviepy - Writing video I:\乐高\图纸\002长颈鹿\002长颈鹿_搭建视频.mp4



Moviepy - Done !
Moviepy - video ready I:\乐高\图纸\002长颈鹿\002长颈鹿_搭建视频.mp4
Done


In [ ]:
# 自动剪辑【秒懂一分钟视频】

import os
from moviepy.editor import *

class video:
    def __init__(self,name):
        self.pth='I:\乐高\图纸'
        self.name=name
        
    def cut(self):
        inputVideoName=os.path.join(self.pth,self.name)
        outVideo=VideoFileClip(inputVideoName).cutout('00:00:00.00','00:00:05.02')
#         finalclip = concatenate_videoclips([outVideo])
            
        outVideo.write_videofile(os.path.join(self.pth,'cut_start.mp4'))
        
if __name__=='__main__':
    v=video('lion.mp4')
    v.cut()


In [2]:
#测试用
import os
from PIL import Image
from PIL import ImageDraw  
from PIL import ImageFont
import re

k='I:\乐高\图纸\002长颈鹿/零件总图/1_1x.png'
p=k.replace('.png','_tagged.png')
print(p)



I:\乐高\图纸长颈鹿/零件总图/1_1x_tagged.png
